In [249]:
import pandas as pd
import numpy as np
from numpy import percentile
import pickle

from math import sqrt
import statistics
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
import time

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import pymssql
import pandas.io.sql as psql

In [8]:
connection = pymssql.connect(server='10.0.29.132\dev',
                             user='dsread',
                             password='dsread#2020',
                             database='AAGL10S_Dev')

In [9]:
cursor = connection.cursor()
fd=pd.read_sql_query("select * from powerdailyusage du where du.AMIMeterNumber in(select top(100) AMIMeterNumber from powerdailyusage group by AMIMeterNumber order by count(*) desc) order by du.AMIMeternumber,du.UsageDate",connection)

100 AMIMeter Numbers are been taken for the analysis

In [10]:
fd

,DailyUsagesId,AccountNumber,Usagedate,Value,Consumed,MeterId,CreatedDate,UpdatedDate,RatePlanDetailId,AMIMeterNumber,attribute1,attribute2,attribute3,attribute4,attribute5
0,5,None,2020-08-01,0.0000,NaN,None,2021-06-18 12:20:30.747,2021-06-18 12:20:30.747,NaN,E00050491,5,1.00,,load_usage1,None
1,239,None,2020-08-01,0.0000,NaN,None,2021-06-18 12:20:30.747,2021-06-18 12:20:30.747,NaN,E00050491,5,1.00,,load_usage2,None
2,12257873,None,2020-08-01,8.9799,NaN,None,2021-06-24 13:18:08.280,2021-06-24 12:44:27.260,NaN,E00050491,5,1.00,off peak,Usage,None
3,240,None,2020-08-01,10.4107,NaN,None,2021-06-18 12:20:30.747,2021-06-18 12:20:30.747,NaN,E00050491,5,1.00,peak,Usage,None
4,6,None,2020-08-01,9.7541,NaN,None,2021-06-18 12:20:30.747,2021-06-18 12:20:30.747,NaN,E00050491,5,1.00,shoulder,Usage,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19795,11923403,None,2020-09-10,8.7487,NaN,None,2021-06-19 00:29:53.890,2021-06-19 00:29:53.890,NaN,E02AC9373,5,1.00,peak,Usage,None
19796,11923404,None,2020-09-10,10.3041,NaN,None,2021-06-19 00:29:53.890,2021-06-19 00:29:53.890,NaN,E02AC9373,5,1.00,shoulder,Usage,None
19797,15402686,None,2020-09-11,0.0000,NaN,None,2021-07-03 00:53:12.020,2021-07-02 23:39:24.190,NaN,E02AC9373,5,1.00,,load_usage1,None
19798,15402687,None,2020-09-11,0.0000,NaN,None,2021-07-03 00:53:12.020,2021-07-02 23:39:24.190,NaN,E02AC9373,5,1.00,,load_usage2,None


In [ ]:
#cursor = connection.cursor()
#df= pd.read_sql_query("select amm.AccountNumber, amm.MeterId, du.* from powerDailyUsage du join AccountMeterMapping amm on du.AMIMeterNumber = amm.AMIMeterNumber", connection)

In [11]:
fd['AMIMeterNumber'].value_counts()

E01A7CB91    198
E0021CD36    198
E00851C19    198
E01717A94    198
E02A735D7    198
            ... 
E016C8FFE    198
E000B8F47    198
E01C33A67    198
E01CB83F6    198
E00790F71    198
Name: AMIMeterNumber, Length: 100, dtype: int64

In [26]:
main_agl100=fd[fd['attribute4']=='Usage']

In [27]:
agl100=main_agl100.drop(columns=['CreatedDate','UpdatedDate','RatePlanDetailId','Consumed','MeterId','AccountNumber'],axis=1)

In [28]:
agl100

,DailyUsagesId,Usagedate,Value,AMIMeterNumber,attribute1,attribute2,attribute3,attribute4,attribute5
2,12257873,2020-08-01,8.9799,E00050491,5,1.00,off peak,Usage,None
3,240,2020-08-01,10.4107,E00050491,5,1.00,peak,Usage,None
4,6,2020-08-01,9.7541,E00050491,5,1.00,shoulder,Usage,None
7,12361418,2020-08-02,12.3264,E00050491,5,1.00,off peak,Usage,None
8,316031,2020-08-02,9.3859,E00050491,5,1.00,peak,Usage,None
...,...,...,...,...,...,...,...,...,...
19791,15662397,2020-09-08,7.0000,E02AC9373,5,1.00,shoulder,Usage,None
19794,15167589,2020-09-10,8.0000,E02AC9373,5,1.00,off peak,Usage,None
19795,11923403,2020-09-10,8.7487,E02AC9373,5,1.00,peak,Usage,None
19796,11923404,2020-09-10,10.3041,E02AC9373,5,1.00,shoulder,Usage,None


# data is been reshaped in the form of components A,B,C that are offpeak,peak,shoulder respectively.

In [87]:
x=agl100.pivot_table('Value',['Usagedate','AMIMeterNumber'],'attribute3')
x.reset_index(drop=False,inplace=True)
x.reindex(['Usagedate','AMIMeterNumber','off peak','peak','shoulder'],axis=1)
x=x.rename(index={'attribute3':'index'})
x

attribute3,Usagedate,AMIMeterNumber,off peak,peak,shoulder
0,2020-08-01,E00050491,8.9799,10.4107,9.7541
1,2020-08-01,E000B8207,9.6568,9.1095,7.9840
2,2020-08-01,E000B8F47,10.4962,8.1234,9.0979
3,2020-08-01,E0010CC0F,7.7093,9.8410,9.2023
4,2020-08-01,E001402B1,9.8602,10.0279,9.6277
...,...,...,...,...,...
4095,2020-09-11,E029B55D7,9.0000,NaN,NaN
4096,2020-09-11,E029F9CDE,8.0000,NaN,NaN
4097,2020-09-11,E02A63505,10.0000,NaN,NaN
4098,2020-09-11,E02A735D7,8.0000,NaN,NaN


In [100]:
x['AMIMeterNumber'].unique()

array(['E00050491', 'E000B8207', 'E000B8F47', 'E0010CC0F', 'E001402B1',
       'E001E1672', 'E001EF59F', 'E0021CD36', 'E0022E832', 'E0028608C',
       'E00289C80', 'E002A69BF', 'E002BA7CD', 'E003659CF', 'E0037D0A4',
       'E003BA0A3', 'E0044201D', 'E004E41FF', 'E005EF814', 'E00621C19',
       'E006285FB', 'E0069BE5D', 'E006A6A43', 'E0074DD34', 'E00753C7D',
       'E00790F71', 'E00809B96', 'E00851C19', 'E0092BAF6', 'E00953DE2',
       'E009FD3BD', 'E00AD63BA', 'E00B07B22', 'E00B890B0', 'E00BF3251',
       'E00C35818', 'E00D34797', 'E00FD1AFA', 'E01035E15', 'E0107F8AF',
       'E010BF82D', 'E011460E9', 'E0124A86A', 'E01252F44', 'E01418580',
       'E0148E945', 'E014A83AF', 'E0152BF13', 'E015721D0', 'E0162F37F',
       'E016C8FFE', 'E01717A94', 'E01773EA5', 'E0178F3D2', 'E0180F90C',
       'E01823014', 'E01854234', 'E01920338', 'E01941C9C', 'E0195B794',
       'E01A34277', 'E01A4CD37', 'E01A5972B', 'E01A5BB1C', 'E01A77097',
       'E01A7CB91', 'E01B2A109', 'E01BC8902', 'E01C1B560', 'E01C

In [114]:
x[x.peak.isnull()]

attribute3,Usagedate,AMIMeterNumber,off peak,peak,shoulder
3600,2020-09-06,E00050491,8.0,NaN,11.5260
3601,2020-09-06,E000B8207,9.0,NaN,5.3916
3602,2020-09-06,E000B8F47,11.0,NaN,9.5700
3603,2020-09-06,E0010CC0F,9.0,NaN,12.2669
3604,2020-09-06,E001402B1,7.0,NaN,7.1383
...,...,...,...,...,...
4095,2020-09-11,E029B55D7,9.0,NaN,NaN
4096,2020-09-11,E029F9CDE,8.0,NaN,NaN
4097,2020-09-11,E02A63505,10.0,NaN,NaN
4098,2020-09-11,E02A735D7,8.0,NaN,NaN


In [120]:
percent_missing=x.isnull().sum()*100/len(x)

In [117]:
percent_missing

attribute3
Usagedate         0.000000
AMIMeterNumber    0.000000
off peak          7.317073
peak              7.317073
shoulder          2.439024
dtype: float64

In [93]:
x[x['off peak'].isna()]

attribute3,Usagedate,AMIMeterNumber,off peak,peak,shoulder
2900,2020-08-30,E00050491,NaN,8.4439,7.0378
2901,2020-08-30,E000B8207,NaN,10.8484,6.5294
2902,2020-08-30,E000B8F47,NaN,7.3644,10.9982
2903,2020-08-30,E0010CC0F,NaN,10.2372,9.8906
2904,2020-08-30,E001402B1,NaN,8.7811,8.3721
...,...,...,...,...,...
3895,2020-09-08,E029B55D7,NaN,NaN,4.0000
3896,2020-09-08,E029F9CDE,NaN,NaN,5.0000
3897,2020-09-08,E02A63505,NaN,NaN,4.0000
3898,2020-09-08,E02A735D7,NaN,NaN,5.0000


In [94]:
x[x['shoulder'].isna()]

attribute3,Usagedate,AMIMeterNumber,off peak,peak,shoulder
4000,2020-09-11,E00050491,8.0,NaN,NaN
4001,2020-09-11,E000B8207,11.0,NaN,NaN
4002,2020-09-11,E000B8F47,10.0,NaN,NaN
4003,2020-09-11,E0010CC0F,5.0,NaN,NaN
4004,2020-09-11,E001402B1,11.0,NaN,NaN
...,...,...,...,...,...
4095,2020-09-11,E029B55D7,9.0,NaN,NaN
4096,2020-09-11,E029F9CDE,8.0,NaN,NaN
4097,2020-09-11,E02A63505,10.0,NaN,NaN
4098,2020-09-11,E02A735D7,8.0,NaN,NaN


In [54]:
x.describe()

attribute3,off peak,peak,shoulder
count,3800.000000,3800.000000,4000.000000
mean,8.862730,9.533022,9.459018
std,1.777409,1.595652,1.743308
min,3.000000,4.116600,2.000000
25%,7.834650,8.445100,8.397475
50%,9.000000,9.549550,9.513800
75%,10.000000,10.638875,10.627350
max,15.456900,15.315900,15.398000


In [57]:
x['peak'].isna().sum()

300

In [59]:
x['off peak'].isna().sum()

300

In [60]:
x['shoulder'].isna().sum()

100

In [61]:
x.isnull().values.any()

True

In [74]:
x.count()

attribute3
Usagedate         4100
AMIMeterNumber    4100
off peak          3800
peak              3800
shoulder          4000
dtype: int64

In [20]:
agl100['AMIMeterNumber'].value_counts()

E01A7CB91    116
E0021CD36    116
E00851C19    116
E01717A94    116
E02A735D7    116
            ... 
E016C8FFE    116
E000B8F47    116
E01C33A67    116
E01CB83F6    116
E00790F71    116
Name: AMIMeterNumber, Length: 100, dtype: int64

In [146]:
single_acc=x[x.AMIMeterNumber=='E00050491']
single_acc=single_acc.sort_values(by='Usagedate')
single_acc

attribute3,Usagedate,AMIMeterNumber,off peak,peak,shoulder
0,2020-08-01,E00050491,8.9799,10.4107,9.7541
100,2020-08-02,E00050491,12.3264,9.3859,11.6094
200,2020-08-03,E00050491,11.9143,8.0572,8.7935
300,2020-08-04,E00050491,9.8681,8.1949,10.1372
400,2020-08-05,E00050491,9.2135,8.8307,8.0599
500,2020-08-06,E00050491,8.8518,8.6421,6.5967
600,2020-08-07,E00050491,12.1815,9.1029,9.5123
700,2020-08-08,E00050491,9.5030,10.0991,10.8692
800,2020-08-09,E00050491,9.5476,8.9434,8.6563
900,2020-08-10,E00050491,7.7541,9.0569,5.9899


In [225]:
meter_list =x.AMIMeterNumber.unique()

# percent_missing NULL values

In [226]:
def replacing(met):
        sin=x[x.AMIMeterNumber==met]
        sin=sin.sort_values('Usagedate')
        fin=sin.interpolate(method='linear')
        return fin
    

In [227]:
fg=[]
for i in meter_list:
        res=replacing(i)
        fg.append(res)
fg=pd.concat(fg)
fg
        

attribute3,Usagedate,AMIMeterNumber,off peak,peak,shoulder
0,2020-08-01,E00050491,8.9799,10.41070,9.7541
100,2020-08-02,E00050491,12.3264,9.38590,11.6094
200,2020-08-03,E00050491,11.9143,8.05720,8.7935
300,2020-08-04,E00050491,9.8681,8.19490,10.1372
400,2020-08-05,E00050491,9.2135,8.83070,8.0599
...,...,...,...,...,...
3699,2020-09-06,E02AC9373,10.0000,9.66970,8.7598
3799,2020-09-07,E02AC9373,8.0000,9.27180,12.3588
3899,2020-09-08,E02AC9373,8.0000,9.01025,7.0000
3999,2020-09-10,E02AC9373,8.0000,8.74870,10.3041


In [228]:
fg=fg.sort_values('Usagedate')
fg

attribute3,Usagedate,AMIMeterNumber,off peak,peak,shoulder
0,2020-08-01,E00050491,8.9799,10.4107,9.7541
36,2020-08-01,E00D34797,8.7805,7.0581,11.7213
88,2020-08-01,E02559602,9.8403,8.8778,9.6528
4,2020-08-01,E001402B1,9.8602,10.0279,9.6277
46,2020-08-01,E014A83AF,8.7337,9.6581,7.8736
...,...,...,...,...,...
4068,2020-09-11,E01C1B560,6.0000,7.6935,10.7324
4069,2020-09-11,E01C33A67,7.0000,9.0470,11.1045
4070,2020-09-11,E01C421A2,10.0000,10.8356,8.7850
4047,2020-09-11,E0152BF13,11.0000,7.9607,8.9704


In [229]:
fg['RatePlanDetailId']=1

In [230]:
fg

attribute3,Usagedate,AMIMeterNumber,off peak,peak,shoulder,RatePlanDetailId
0,2020-08-01,E00050491,8.9799,10.4107,9.7541,1
36,2020-08-01,E00D34797,8.7805,7.0581,11.7213,1
88,2020-08-01,E02559602,9.8403,8.8778,9.6528,1
4,2020-08-01,E001402B1,9.8602,10.0279,9.6277,1
46,2020-08-01,E014A83AF,8.7337,9.6581,7.8736,1
...,...,...,...,...,...,...
4068,2020-09-11,E01C1B560,6.0000,7.6935,10.7324,1
4069,2020-09-11,E01C33A67,7.0000,9.0470,11.1045,1
4070,2020-09-11,E01C421A2,10.0000,10.8356,8.7850,1
4047,2020-09-11,E0152BF13,11.0000,7.9607,8.9704,1


In [248]:
fg.describe()

attribute3,off peak,peak,shoulder,RatePlanDetailId
count,4100.000000,4100.000000,4100.000000,4100.0
mean,8.806920,9.538408,9.469182,1.0
std,1.758712,1.574262,1.738943,0.0
min,3.000000,4.116600,2.000000,1.0
25%,7.721625,8.463400,8.414150,1.0
50%,8.999750,9.550550,9.522750,1.0
75%,10.000000,10.635925,10.637125,1.0
max,15.456900,15.315900,15.398000,1.0


In [19]:
single_acc=agl100[agl100.AMIMeterNumber=='E00050491']
single_acc=single_acc.sort_values(by='Usagedate')
single_acc

,DailyUsagesId,Usagedate,Value,AMIMeterNumber,attribute1,attribute2,attribute3,attribute4,attribute5
2,12257873,2020-08-01,8.9799,E00050491,5,1.00,off peak,Usage,None
3,240,2020-08-01,10.4107,E00050491,5,1.00,peak,Usage,None
4,6,2020-08-01,9.7541,E00050491,5,1.00,shoulder,Usage,None
7,12361418,2020-08-02,12.3264,E00050491,5,1.00,off peak,Usage,None
8,316031,2020-08-02,9.3859,E00050491,5,1.00,peak,Usage,None
...,...,...,...,...,...,...,...,...,...
189,15563928,2020-09-08,7.0000,E00050491,5,1.00,shoulder,Usage,None
193,11921310,2020-09-10,12.0197,E00050491,5,1.00,peak,Usage,None
194,11921311,2020-09-10,13.1395,E00050491,5,1.00,shoulder,Usage,None
192,15166767,2020-09-10,9.0000,E00050491,5,1.00,off peak,Usage,None


In [103]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [104]:
def amimeter_month(dataset,amimeterid):
    df2 = dataset[dataset.AMIMeterNumber==amimeterid]
    a = diff_month(df2['Usagedate'].iloc[-1],df2['Usagedate'].iloc[0])
    return a

In [106]:
amimeter_month(x,'E000B8F47')

1

In [107]:
def total_months1(dataset):
    d = pd.DataFrame()
    amimeter_list = dataset.AMIMeterNumber.unique()
    for i in amimeter_list:
        d.loc[i,['TotalMonths']] = amimeter_month(dataset,i)
    d = d.reset_index()
    d.rename(columns = {'index':'amiMeterId'},inplace=True)
    return d

In [108]:
f1 = total_months1(x)

In [110]:
f1

,amiMeterId,TotalMonths
0,E00050491,1.0
1,E000B8207,1.0
2,E000B8F47,1.0
3,E0010CC0F,1.0
4,E001402B1,1.0
...,...,...
95,E029B55D7,1.0
96,E029F9CDE,1.0
97,E02A63505,1.0
98,E02A735D7,1.0


In [109]:
f1['TotalMonths'].value_counts()

1.0    100
Name: TotalMonths, dtype: int64

In [231]:
def mape_new(yTrue,yPredicted):
    return np.abs((yTrue - yPredicted) / yTrue) * 100

In [232]:
def compA(i, fg, testdays):
    
            meterID_df = fg[fg.AMIMeterNumber == i]
            meterID_df = meterID_df.drop(['off peak','shoulder','AMIMeterNumber'],axis=1)
            final = meterID_df.reset_index()[['Usagedate','peak']].rename({'Usagedate':'ds','peak':'y'}, axis='columns')
            train = final[:len(final)-testdays] # training data
            original = final[len(final)-testdays:]
            test = final[len(final)-testdays:] # test data
            test_remaining = original[:len(original)-testdays]
            sum_test_remaining = test_remaining['y'].sum()
            sum_actual = original['y'].sum()
            p = Prophet(daily_seasonality = False, weekly_seasonality = False, yearly_seasonality = True).fit(train) 
            prediction = p.make_future_dataframe(periods=testdays , freq='D')
            fore = p.predict(prediction)
            test['Predictions'] = fore['yhat']
            combined = pd.DataFrame()
            combined['ds'] = original['ds']
            combined['y'] = test['Predictions']
            sum_predictions = test['Predictions'].sum()
            sum_predictions_final = sum_predictions
            MAPE_new = mape_new(sum_actual, sum_predictions_final)
            #overall_sum = overall_sum + sum_predictions_final
            #print(final)
            #print(train)
            #print(test)
            #print()
            #print()
            #print('\033[1m Overall Prediction Sum: %.3f \033[1m' % overall_sum)
            #p.plot(fore)
            #plt.title('Prediction plot of ' +str(testdays)+ ' days for meter Number: ' +str(met))
            #print('\033[1m Mape error: %.3f \033[1m' % MAPE_new)
            return sum_predictions_final

In [233]:
compA('E000B8F47',fg,7)

/Users/nikhilmylarusetty/opt/anaconda3/envs/Super/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


175.56873779935668

In [234]:
def compB(i, fg, testdays):
    
            meterID_df = fg[fg.AMIMeterNumber == i]
            meterID_df = meterID_df.drop(['peak','shoulder','AMIMeterNumber'],axis=1)
            final = meterID_df.reset_index()[['Usagedate','off peak']].rename({'Usagedate':'ds','off peak':'y'}, axis='columns')
            train = final[:len(final)-testdays] # training data
            original = final[len(final)-testdays:]
            test = final[len(final)-testdays:] # test data
            test_remaining = original[:len(original)-testdays]
            sum_test_remaining = test_remaining['y'].sum()
            sum_actual = original['y'].sum()
            p = Prophet(daily_seasonality = False, weekly_seasonality = False, yearly_seasonality = True).fit(train) 
            prediction = p.make_future_dataframe(periods=testdays , freq='D')
            fore = p.predict(prediction)
            test['Predictions'] = fore['yhat']
            combined = pd.DataFrame()
            combined['ds'] = original['ds']
            combined['y'] = test['Predictions']
            sum_predictions = test['Predictions'].sum()
            sum_predictions_final = sum_predictions
            MAPE_new = mape_new(sum_actual, sum_predictions_final)
            #overall_sum = overall_sum + sum_predictions_final
            #print(final)
            #print(train)
            #print(test)
            #print()
            #print()
            #print('\033[1m Overall Prediction Sum: %.3f \033[1m' % overall_sum)
            #p.plot(fore)
            #plt.title('Prediction plot of ' +str(testdays)+ ' days for meter Number: ' +str(met))
            #print('\033[1m Mape error: %.3f \033[1m' % MAPE_new)
            return sum_predictions_final

In [235]:
compB('E000B8F47',fg,7)

/Users/nikhilmylarusetty/opt/anaconda3/envs/Super/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


73.57762568515622

In [236]:
def compC(i, fg, testdays):
    
            meterID_df = fg[fg.AMIMeterNumber == i]
            meterID_df = meterID_df.drop(['peak','off peak','AMIMeterNumber'],axis=1)
            final = meterID_df.reset_index()[['Usagedate','shoulder']].rename({'Usagedate':'ds','shoulder':'y'}, axis='columns')
            train = final[:len(final)-testdays] # training data
            original = final[len(final)-testdays:]
            test = final[len(final)-testdays:] # test data
            test_remaining = original[:len(original)-testdays]
            sum_test_remaining = test_remaining['y'].sum()
            sum_actual = original['y'].sum()
            p = Prophet(daily_seasonality = False, weekly_seasonality = False, yearly_seasonality = True).fit(train) 
            prediction = p.make_future_dataframe(periods=testdays , freq='D')
            fore = p.predict(prediction)
            test['Predictions'] = fore['yhat']
            combined = pd.DataFrame()
            combined['ds'] = original['ds']
            combined['y'] = test['Predictions']
            sum_predictions = test['Predictions'].sum()
            sum_predictions_final = sum_predictions
            MAPE_new = mape_new(sum_actual, sum_predictions_final)
            #overall_sum = overall_sum + sum_predictions_final
            #print(final)
            #print(train)
            #print(test)
            #print()
            #print()
            #print('\033[1m Overall Prediction Sum: %.3f \033[1m' % overall_sum)
            #p.plot(fore)
            #plt.title('Prediction plot of ' +str(testdays)+ ' days for meter Number: ' +str(met))
            #print('\033[1m Mape error: %.3f \033[1m' % MAPE_new)
            return sum_predictions_final

In [237]:
compC('E000B8F47',fg,7)

/Users/nikhilmylarusetty/opt/anaconda3/envs/Super/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


91.0807886448476

In [246]:

def account_and_days(met, testdays):
        total_compA_usage = 0    
        total_compA_bill = 0
        total_compB_usage = 0   
        total_compB_bill = 0
        total_compC_usage = 0
        compCbill=0              
        total_compC_bill = 0
        tier1_rate=10
        tier2_rate=20
        threshold=70
        rate = 15
        supply_charge = 50
    
        #meterID_list = pd.DataFrame()
        #meterID_list = fg[fg.AMIMeterNumber == met]
        #for i in meterID_list:
        
        compAusage = compA(met,fg, testdays)
        print('Component A usage for Meter ID ',met,': %.3f' % compAusage)
        total_compA_usage = total_compA_usage + compAusage
        
        compAbill = (compAusage * rate) + supply_charge
        print('Component A bill for Meter ID ',met,': $ %.3f' % compAbill)
        total_compA_bill = total_compA_bill + compAbill
        
        print()
        
        compBusage = compB(met,fg,testdays)
        print('Component B usage for Meter ID ',met,': %.3f' % compBusage)
        total_compB_usage = total_compB_usage + compBusage
        
        compBbill = (compBusage * rate) + supply_charge
        print('Component B bill for Meter ID ',met,': $ %.3f' % compBbill)
        total_compB_bill = total_compB_bill + compBbill
        print()
        x = fg[fg.AMIMeterNumber == met].RatePlanDetailId
        res = x.to_frame().rename({'RatePlanDetailId':'RatePlan'}, axis='columns')
        if ((res['RatePlan']==1).all()) :
                compCusage = compC(met, fg, testdays)
                print('Component C usage for Meter ID ',met,': %.3f' % compCusage)
                total_compC_usage = total_compC_usage + compCusage

                compCbill = (compCusage * rate) + supply_charge
                print('Component C bill for Meter ID ',met,': $ %.3f' % compCbill)
                total_compC_bill = total_compC_bill + compCbill
        elif ((res['RatePlan']==2).all()) :
                compCusage = compC(met, fg, testdays)
                print('Component C usage for Meter ID ',met,': %.3f' % compCusage)
                total_compC_usage = total_compC_usage + compCusage
                
                if (total_compC_usage <=  threshold):
                    compCbill = compCbill + (compCusage * tier1_rate ) + supply_charge
                elif (total_compC_usage >  threshold):
                    compCbill = compCbill + (compCusage * tier2_rate ) + supply_charge
                
                print('Component C bill for Meter ID ',met,': $ %.3f' % compCbill)
                total_compC_bill = total_compC_bill + compCbill   
        print()
        
    

In [247]:
account_and_days('E00050491', 7)

/Users/nikhilmylarusetty/opt/anaconda3/envs/Super/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Component A usage for Meter ID  E00050491 : 50.779
Component A bill for Meter ID  E00050491 : $ 811.683



/Users/nikhilmylarusetty/opt/anaconda3/envs/Super/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Component B usage for Meter ID  E00050491 : 68.675
Component B bill for Meter ID  E00050491 : $ 1080.127

Component C usage for Meter ID  E00050491 : 107.236
Component C bill for Meter ID  E00050491 : $ 1658.541



/Users/nikhilmylarusetty/opt/anaconda3/envs/Super/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
